# Problem 0: Soccer Guru

_Version 1.5_

Soccer season is on and teams need to start preparing for the World Cup 2022. We need your help as a **Soccer Guru** to analyse different statistics and come up with insights to help the teams prepare better.

This problem tests your understanding of Pandas and SQL concepts.

**Important note.** Due to a limitation in Vocareum's software stack, this notebook is set to use the Python 3.5 kernel (rather than a more up-to-date 3.6 or 3.7 kernel). If you are developing on your local machine and are using a different version of Python, you may need to adapt your solution before submitting to the autograder.


**Exercise 0** (0 points). Run the code cell below to load the data, which is a SQLite3 database containing results and fixtures of various soccer matches that have been played around the globe since 1980.

Observe that the code loads all rows from the table, `soccer_results`, contained in the database file, `prob0.db`.

> You do not need to do anything for this problem other than run the next two code cells and familiarize yourself with the resulting dataframe, which is stored in the variable `df`.

In [1]:
import sqlite3 as db
import pandas as pd
from datetime import datetime
from collections import defaultdict
disk_engine = db.connect('file:prob0.db?mode=ro', uri=True)

def load_data():
    df = pd.read_sql_query("SELECT * FROM soccer_results", disk_engine) 
    return df

In [2]:
# Test: Exercise 0 (exposed)
df = load_data()
assert df.shape[0] == 22851, "Row counts do not match. Try loading the data again"
assert df.shape[1] == 9, "You don't have all the columns. Try loading the data again"
print("\n(Passed!)")
df.head()


(Passed!)


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1994-01-02,Barbados,Grenada,0,0,Friendly,Bridgetown,Barbados,FALSE
1,1994-01-02,Ghana,Egypt,2,1,Friendly,Accra,Ghana,FALSE
2,1994-01-05,Mali,Burkina Faso,1,1,Friendly,Bamako,Mali,FALSE
3,1994-01-09,Mauritania,Mali,1,3,Friendly,Nouakchott,Mauritania,FALSE
4,1994-01-11,Thailand,Nigeria,1,1,Friendly,Bangkok,Thailand,FALSE


Each row of this dataframe is a game, which is played between a "home team" (column `home_team`) and an "away team" (`away_team`). The number of goals scored by each team appears in the `home_score` and `away_score` columns, respectively.

**Exercise 1** (1 point): Write an **SQL query** find the ten (10) teams that have the highest average away-scores since the year 2000. Your query should satisfy the following criteria:

- It should return two columns:
    * `team`: The name of the team
    * `ave_goals`: The team's average number of goals **in "away" games.** An "away game" is one in which the team's name appars in `away_team` **and** the game takes place at a "non-neutral site" (`neutral` value equals `FALSE`).
- It should only include teams that have played **at least 30 away matches**.
- It should round the average goals value (`ave_goals`) to three decimal places.
- It should only return the top 10 teams in descending order by average away-goals.
- It should only consider games played since 2000 (including the year 2000).

Store your query string as the variable, `query_top10_away`, below. The test cell will run this query string against the input dataframe, `df`, defined above and return the result in a dataframe named `offensive_teams`. (See the test cell.)

> **Note.** The following exercises have hidden test cases and you'll be awarded full points for passing both the exposed and hidden test cases.

In [3]:
c = disk_engine.cursor()

query_top10_away = '''
    
    with two as (
    with one as (
    Select away_team as team, count(*) as matches
    from soccer_results
    where neutral = 'FALSE' 
    group by team
    order by matches desc)
    select team 
    from one 
    where matches > 29)
    
    Select away_team as team, round(avg(away_score),3) as ave_goals
    from soccer_results
    where neutral = 'FALSE' and cast(strftime('%Y',date) as int) > 1999 and team in two
    group by team
    order by ave_goals desc
    Limit 10

'''  # Write your query here!



c.execute(query_top10_away)
result = c.fetchall()
result
#print(query_top10_away)

[('Germany', 2.17),
 ('Brazil', 2.01),
 ('Spain', 1.927),
 ('England', 1.763),
 ('Netherlands', 1.742),
 ('France', 1.639),
 ('Portugal', 1.579),
 ('Argentina', 1.56),
 ('Saudi Arabia', 1.54),
 ('Denmark', 1.534)]

In [4]:
# Test: Exercise 1 (exposed)
offensive_teams = pd.read_sql_query(query_top10_away, disk_engine)
df_cols = offensive_teams.columns.tolist()
df_cols.sort()
desired_cols = ['team', 'ave_goals']
desired_cols.sort()
print(offensive_teams.head(10))
assert offensive_teams.shape[0] == 10, "Expected 10 rows but returned dataframe has {}".format(offensive_teams.shape[0])
assert offensive_teams.shape[1] == 2, "Expected 2 columns but returned dataframe has {}".format(offensive_teams.shape[1])
assert df_cols == desired_cols, "Column names should be: {}. Returned dataframe has: {}".format(desired_cols, df_cols)

tolerance = .001
team_4 = offensive_teams.iloc[3].team
team_4_ave = offensive_teams.iloc[3].ave_goals
desired_team_4_ave = 1.763
assert (team_4 == "England" and abs(team_4_ave - 1.763) <= .001), "Fourth entry is {} with average of {}. Got {} with average of {}".format("England", 1.76, team_4, team_4_ave)

print("\n(Passed!)")

           team  ave_goals
0       Germany      2.170
1        Brazil      2.010
2         Spain      1.927
3       England      1.763
4   Netherlands      1.742
5        France      1.639
6      Portugal      1.579
7     Argentina      1.560
8  Saudi Arabia      1.540
9       Denmark      1.534

(Passed!)


In [5]:
# Hidden test cell: exercise1_hidden

print("""
In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.
""")

###
### AUTOGRADER TEST - DO NOT REMOVE
###



In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.



**Exercise 2** (2 points): Suppose we are now interested in the top 10 teams having the best goal **differential**, between the years 2012 and 2018 (both inclusive). A team's goal differential is the difference between the total number of goals it scored and the total number it conceded across all games (in the requested years).

Complete the function, `best_goal_differential()`, below, so that it returns a pandas dataframe containing the top 10 teams by goal differential, sorted in descending order of differential. The dataframe should have two columns: `team`, which holds the team's name, and `differential`, which holds its overall goal differential.

> As a sanity check, you should find the Brazil is the number one team, with a differential of 152 during the selected time period of 2012-2018 (inclusive). It should be the first row of the returned dataframe.

In [6]:
def best_goal_differential():
    query_top_diff = '''
    
        with home as (
        select home_team as team, sum(home_score) as score, sum(away_score) as concede
        from soccer_results
        where cast(strftime('%Y',date) as int) > 2011 and cast(strftime('%Y',date) as int) < 2019
        group by team
        )
        
        , away as (
        select away_team as team, sum(away_score) as score, sum(home_score) as concede
        from soccer_results
        where cast(strftime('%Y',date) as int) > 2011 and cast(strftime('%Y',date) as int) < 2019
        group by team
        )

        Select home.team as team, away.score+home.score-home.concede-away.concede as differential
        from home, away
        where home.team = away.team
        order by differential desc
        limit 10

    '''

    goal_diff = pd.read_sql_query(query_top_diff, disk_engine)
    return goal_diff

best_goal_differential()


,team,differential
0,Brazil,152
1,Spain,147
2,Belgium,119
3,Germany,113
4,France,98
5,Iran,91
6,England,90
7,Portugal,87
8,Argentina,86
9,Japan,81


In [7]:
# Test: Exercise 2 (exposed)

diff_df = best_goal_differential()
df_cols = diff_df.columns.tolist()
df_cols.sort()
desired_cols = ['team', 'differential']
desired_cols.sort()

assert isinstance(diff_df, pd.DataFrame), "Dataframe object not returned"
assert diff_df.shape[0] == 10, "Expected 10 rows but returned dataframe has {}".format(diff_df.shape[0])
assert diff_df.shape[1] == 2, "Expected 2 columns but returned dataframe has {}".format(diff_df.shape[1])
assert df_cols == desired_cols, "Column names should be: {}. Returned dataframe has: {}".format(desired_cols, df_cols)

best_team = diff_df.iloc[0].team
best_diff = diff_df.iloc[0].differential
assert (best_team == "Brazil" and best_diff == 152), "{} has best differential of {}. Got team {} having best differential of {}".format("Brazil", 152, best_team, best_diff)

print("\n(Passed!)")


(Passed!)


In [8]:
# Hidden test cell: exercise2_hidden

print("""
In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.
""")

###
### AUTOGRADER TEST - DO NOT REMOVE
###



In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.



**Exercise 3** (1 point). Complete the function, `determine_winners(game_df)`, below. It should determine the winner of each soccer game.

In particular, the function should take in a dataframe like `df` from above. It should return a new dataframe consisting of all the columns from that dataframe plus a new columnn called **`winner`**, holding the name of the winning team. If there is no winner for a particular game (i.e., the score is tied), then the `winner` column should containing the string, `'Draw'`. Lastly, the rows of the output should be in the same order as the input dataframe.

You can use any dataframe manipulation techniques you want for this question _(i.e., pandas methods or SQL queries, as you prefer)._

> You'll need the output dataframe from this exercise for the subsequent exercies, so don't skip this one!

In [9]:
import numpy as np
def determine_winners(game_df):
    
    conditions = [ (game_df['home_score'] > df['away_score']), game_df['away_score'] > game_df['home_score']]

    choices = [game_df['home_team'], game_df['away_team']]

    game_df['winner'] = np.select(conditions, choices, default='Draw')
    
    return game_df

In [10]:
# Test: Exercise 3 (exposed)

game_df = pd.read_sql_query("SELECT * FROM soccer_results", disk_engine)
winners_df = determine_winners(game_df)

game_winner = winners_df.iloc[1].winner
assert game_winner == "Ghana", "Expected Ghana to be winner. Got {}".format(game_winner)

game_winner = winners_df.iloc[2].winner
assert game_winner == "Draw", "Match was Draw. Got {}".format(game_winner)

game_winner = winners_df.iloc[3].winner
assert game_winner == "Mali", "Expected Mali to be winner. Got {}".format(game_winner)

print("\n(Passed!)")


(Passed!)


In [11]:
# Hidden test cell: exercise3_hidden

print("""
In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.
""")

###
### AUTOGRADER TEST - DO NOT REMOVE
###



In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.



**Exercise 4** (3 points): Given a team, its _home advantage ratio_ is the number of home games it has won divided by the number of home games it has played. For this exercise, we'll try to answer the question, how important is the home advantage in soccer? It's importance is factored into draws for competitions, for example, teams wanting to play at home the second leg of the matches of great importance such as tournament knockouts. (_This exercise has a pre-requisite of finishing Exercise 3 as we'll be using the results of the dataframe from that exercise in this one._)

Complete the function, `calc_home_advantage(winners_df)`, below, so that it returns the top 5 countries, among those that have played at least 50 **home** games, having the highest home advantage ratio. It should return a dataframe with two columns, **`team`** and **`ratio`**, holding the name of the team and its home advantage ratio, respectively. The ratio should be rounded to three decimal places. The rows should be sorted in descending order of ratio. If there are two teams with the same winning ratio, the teams should appear in alphabetical order by name.

> **Note 0.** As with our definition of away-games, a team plays a home game if it is the home team (`home_team`) **and** the field is non-neutral (i.e., `neutral` is `FALSE`).
>
> **Note 1.** You should find, for example, that Brazil is the number two team, with a home advantage ratio of 0.773.

In [12]:
def calc_home_advantage(winners_df):
    
    #removing neutral venues for dataframe
    new_winners_df = winners_df[winners_df['neutral']=='FALSE']
    
    #counting total home games
    homecount_df = new_winners_df.groupby(['home_team'])['neutral'].count().reset_index()
    homecount_df.columns= ['team','home_played']
    
    #separating the countries who played at least 50 games
    homecount_df1 = homecount_df[homecount_df['home_played']>49]
    
    #counting total home wins
    test = new_winners_df
    conditions = [test['home_score'] > test['away_score']]
    choices = [test['home_team']]
    test['home_winner'] = np.select(conditions, choices, default='Draw')
    homewins_df = test.groupby('home_winner')['neutral'].count().reset_index()
    homewins_df.columns= ['team','home_won']
    
    #creating final dataframe with win ratio
    final_df = pd.merge(homecount_df1, homewins_df, on='team')
    final_df['ratio'] = final_df['home_won'] / final_df['home_played']
    final_df1 = final_df[['team', 'ratio']].sort_values(by='ratio',ascending=False)
    final_df1['ratio'] = final_df1['ratio'].round(3)
    final_df1 = final_df1.nlargest(5,'ratio')
    
    return final_df1




In [13]:
# Test: Exercise 4 (exposed)
from IPython.display import display

win_perc = calc_home_advantage(winners_df)

print("The solution, according to you:")
display(win_perc)

df_cols = win_perc.columns.tolist()
df_cols.sort()
desired_cols = ['team', 'ratio']
desired_cols.sort()

assert win_perc.shape[0] == 5, "Expected 5 rows, got {}".format(win_perc.shape[0])
assert win_perc.shape[1] == 2, "Expected 2 columns, got {}".format(win_perc.shape[1])
assert df_cols == desired_cols, "Expected {} columns but got {} columns".format(desired_cols, df_cols)

tolerance = .001
sec_team = win_perc.iloc[1].team
sec_perc = win_perc.iloc[1].ratio

assert (sec_team == "Brazil" and abs(sec_perc - .773) <= tolerance), "Second team should be {} with ratio of {}. \
Got {} with ratio of {}".format("Brazil", .773, sec_team, sec_perc)

print("\n(Passed!)")

The solution, according to you:


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,team,ratio
114,Spain,0.800
18,Brazil,0.773
58,Iran,0.742
21,Cameroon,0.739
34,Egypt,0.724



(Passed!)


In [14]:
# Hidden test cell: exercise4_hidden

print("""
In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.
""")

###
### AUTOGRADER TEST - DO NOT REMOVE
###



In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.



**Exercise 5** (3 points) Now, we've seen how much the home advantage plays in, let us see how the results have looked 
like in the previous tournaments, for the specific case of the FIFA World Cup matches.

In particular, complete the function, `points_table(winners_df, wc_year)`, below, so that it does the following:
- It should take as input a dataframe, `winners_df`, having a "winner" column like that produced in Exercise 3, as well as a target year, `wc_year`.
- It should consider only games in the given target year. Furthermore, it should only consider games where the `tournament` column has the value `"FIFA World Cup"`.
- It should construct and return a "points table". This table should have two columns, **`team`**, containing the team name, and **`points`**, containing a points tally has defined below.
- To compute the points, give the team 3 points for every win, 1 point for every draw, and 0 points (no points) for a loss.
- In case of a tie in the points, sort the teams alphabetically

As an example output, for the 1998 FIFA World Cup, the points table is:

| team        | points |
|-------------|--------|
| France      | 19     |
| Croatia     | 15     |
| Brazil      | 13     |
| Netherlands | 12     |
| Italy       | 11     |

In [18]:
def points_table(winners_df, wc_year):
    
    new_winners_df = winners_df[winners_df['tournament']=='FIFA World Cup']
    
    new_winners_df['date'] = pd.to_datetime(new_winners_df['date'])
    new_winners_df = new_winners_df[new_winners_df['date'].dt.year == wc_year]
    
    conditions = [new_winners_df['winner'] != 'Draw', new_winners_df['winner'] == 'Draw']
    choices = [3,1]
    new_winners_df['points'] = np.select(conditions, choices, default=0)
    
    points_table_wins = new_winners_df.groupby(['winner'])['points'].sum().reset_index()
    points_table_wins.columns= ['team','points_wins']
    
    new_winners_df_draw = new_winners_df[new_winners_df['winner'] == 'Draw']
    points_table_draw1 = new_winners_df_draw.groupby(['home_team'])['points'].sum().reset_index()
    points_table_draw2 = new_winners_df_draw.groupby(['away_team'])['points'].sum().reset_index()
    points_table_draw1.columns= ['team','points1']
    points_table_draw2.columns= ['team','points2']
    
    points_table_draw = pd.merge(points_table_draw1, points_table_draw2, on='team', how='outer')
    points_table_draw['points_draw'] = points_table_draw.fillna(0)['points1']+points_table_draw.fillna(0)['points2']
    points_table_draw = points_table_draw[['team','points_draw']]
    
    points_table = pd.merge(points_table_wins, points_table_draw, on='team',how='outer')
    points_table['points'] = points_table.fillna(0)['points_wins']+points_table.fillna(0)['points_draw']
    points_table = points_table[['team','points']].sort_values(by=['points','team'],ascending=[False, True])
    points_table_final = points_table[points_table['team'] != 'Draw']
    points_table_final = points_table_final.astype({'points': int})
    #points_table_final = points_table_final.nlargest(5,'points')
    
    return points_table_final

points_table(winners_df, 2014)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,team,points
12,Germany,19
17,Netherlands,17
1,Argentina,16
2,Belgium,12
6,Colombia,12
4,Brazil,11
11,France,10
7,Costa Rica,9
5,Chile,7
16,Mexico,7


In [16]:
# Test: Exercise 5 (exposed)


tbl_1998 = points_table(winners_df, 1998)

assert tbl_1998.iloc[0].team == "France"
assert tbl_1998.iloc[0].points == 19
assert tbl_1998.iloc[1].team == "Croatia"
assert tbl_1998.iloc[1].points == 15
assert tbl_1998.iloc[2].team == "Brazil"
assert tbl_1998.iloc[2].points == 13
assert tbl_1998.iloc[3].team == "Netherlands"
assert tbl_1998.iloc[3].points == 12
assert tbl_1998.iloc[4].team == "Italy"
assert tbl_1998.iloc[4].points == 11

print("\n(Passed!)")



(Passed!)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [17]:
# Hidden test cell: exercise5_hidden

print("""
In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.
""")

###
### AUTOGRADER TEST - DO NOT REMOVE
###



In addition to the tests above, this cell will include some hidden tests.
You will only know the result when you submit your solution to the
autograder.



**Fin!** You’ve reached the end of this part. Don’t forget to restart and run all cells again to make sure it’s all working when run in sequence; and make sure your work passes the submission process. Good luck!